In [ ]:
from google.colab import drive
import os


drive.mount('/content/drive')


BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
RAW_DIR = os.path.join(BASE_DIR, 'data/raw')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

if os.path.exists(RAW_DIR):
    print("✅ Conexão bem sucedida!")
    print(f"Arquivos encontrados na pasta RAW: {os.listdir(RAW_DIR)}")
else:
    print(f"❌ Pasta não encontrada: {RAW_DIR}")
    print("Verifique se criou as pastas no Google Drive exatamente com esse nome.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Conexão bem sucedida!
Arquivos encontrados na pasta RAW: ['estimativa_dou_2021.xls', 'estimativa_dou_2014.xls', 'estimativa_dou_2015.xls', 'estimativa_TCU_2016_20170614.xls', 'estimativa_dou_2017.xls', 'estimativa_dou_2019.xls', 'estimativa_dou_2018_20181019.xls', 'estimativa_dou_2024.xls', 'estimativa_dou_2020.xls', 'estimativa_dou_2025.xls']


In [ ]:
    caminho_entrada = os.path.join(RAW_DIR, 'estimativa_dou_2025.xls')
    print(f"--- Processando layout visual: {'estimativa_dou_2025.xls'} ---")
    xls = pd.ExcelFile(caminho_entrada)
    abas_munic = [aba for aba in xls.sheet_names if 'munic' in aba.lower()]
    aba_alvo = abas_munic[0]
    df_raw = pd.read_excel(xls, sheet_name=aba_alvo, header=None)

    print(df_raw)


--- Processando layout visual: estimativa_dou_2025.xls ---
                                                      0        1           2  \
0     ESTIMATIVAS DA POPULAÇÃO RESIDENTE NOS MUNICÍP...      NaN         NaN   
1                                                    UF  COD. UF  COD. MUNIC   
2                                                    RO       11       00015   
3                                                    RO       11       00023   
4                                                    RO       11       00031   
...                                                 ...      ...         ...   
5570                                                 GO       52       22203   
5571                                                 GO       52       22302   
5572                                                 DF       53       00108   
5573                                                NaN      NaN         NaN   
5574  Fonte: IBGE. Diretoria de Pesquisas - DPE -  C...      

In [ ]:
import pandas as pd
import numpy as np
def limpar_valor_populacao(valor):
    """
    Remove formatação, lida com floats (.0) e notas de rodapé.
    """
    # 1. Se já for um número inteiro puro, retorna direto
    if isinstance(valor, (int, np.integer)):
        return int(valor)

    # 2. Se for float (ex: 2982818.0), converte para int (remove o .0)
    if isinstance(valor, (float, np.floating)):
        return int(valor)

    # 3. Se for string, precisamos tratar
    val_str = str(valor).strip()

    # Bugfix: Se terminar com ".0" ou ",0" (comum em conversão de float para str), removemos antes
    if val_str.endswith('.0') or val_str.endswith(',0'):
        val_str = val_str[:-2]

    # Remove tudo que NÃO for dígito (pontos de milhar, parênteses, letras)
    # Ex: "2.982.818(1)" vira "29828181"
    # Melhor estratégia: pegar apenas a primeira sequência de números antes de parênteses
    if '(' in val_str:
        val_str = val_str.split('(')[0]

    # Limpeza final de caracteres não numéricos
    apenas_digitos = ''.join(filter(str.isdigit, val_str))

    if not apenas_digitos:
        return 0 # Valor vazio ou inválido

    return int(apenas_digitos)

# --- FUNÇÃO PRINCIPAL ---
def processar_arquivo_dou(nome_arquivo, ano):
    caminho_entrada = os.path.join(RAW_DIR, nome_arquivo)

    print(f"--- Processando ano {ano}: {nome_arquivo} ---")

    try:
        xls = pd.ExcelFile(caminho_entrada)
        abas_munic = [aba for aba in xls.sheet_names if 'munic' in aba.lower()]

        if not abas_munic:
            return f"❌ Erro: Nenhuma aba de 'Municípios' encontrada."

        aba_alvo = abas_munic[0]
        df_raw = pd.read_excel(xls, sheet_name=aba_alvo, header=None)

    except Exception as e:
        return f"Erro na leitura do Excel: {e}"

    # Busca dinâmica do cabeçalho
    linha_header = -1
    for i in range(15):
        linha = df_raw.iloc[i].astype(str).str.lower().values
        if any('cod' in s for s in linha) and any('uf' in s for s in linha):
            linha_header = i
            break

    if linha_header == -1:
        return "❌ Não achei cabeçalho."

    df = pd.read_excel(xls, sheet_name=aba_alvo, header=linha_header)
    df.columns = [str(c).lower().strip() for c in df.columns]

    try:
        col_cod_uf = [c for c in df.columns if 'uf' in c and 'cod' in c][0]
        col_pop = [c for c in df.columns if 'pop' in c or 'total' in c][0]
    except IndexError:
        return f"Erro colunas."

    # Filtra DF (53)
    df_df = df[df[col_cod_uf].astype(str).str.replace(r'\.0$', '', regex=True) == '53'].copy()

    if df_df.empty:
        return "Erro: DF não encontrado."


    valor_bruto = df_df[col_pop].values[0]
    populacao = limpar_valor_populacao(valor_bruto)

    # Códigos
    cod_7 = '5300108'
    cod_6 = '530010'

    df_final = pd.DataFrame({
        'uf': ['DF'],
        'nome_municipio': ['Brasília'],
        'cod_ibge_7': [cod_7],
        'cod_ibge_6': [cod_6],
        'ano': [ano],
        'populacao': [populacao]
    })

    return df_final


# --- 2. ORQUESTRADOR (Seus arquivos reais) ---

# Mapeamento baseado no seu 'listdir'
arquivos_para_processar = {
    'estimativa_dou_2014.xls': 2014,
    'estimativa_dou_2015.xls': 2015,
    'estimativa_TCU_2016_20170614.xls': 2016, # Nome diferente detectado
    'estimativa_dou_2017.xls': 2017,
    'estimativa_dou_2018_20181019.xls': 2018, # Nome diferente detectado
    'estimativa_dou_2019.xls': 2019,
    'estimativa_dou_2020.xls': 2020,
    'estimativa_dou_2021.xls': 2021,
    # 2022 e 2023 estão faltando na pasta
    'estimativa_dou_2024.xls': 2024,
    'estimativa_dou_2025.xls': 2025
}

lista_dfs = []

print("🚀 Iniciando consolidação...")

for arquivo, ano in arquivos_para_processar.items():
    if os.path.exists(os.path.join(RAW_DIR, arquivo)):
        resultado = processar_arquivo_dou(arquivo, ano)

        if isinstance(resultado, pd.DataFrame):
            lista_dfs.append(resultado)
            print(f"✅ {ano}: Sucesso! População: {resultado['populacao'].values[0]}")
        else:
            print(f"❌ {ano}: Falha -> {resultado}")
    else:
        print(f"⚠️ {ano}: Arquivo '{arquivo}' não encontrado.")

# --- 3. CONSOLIDAÇÃO FINAL ---
if lista_dfs:
    df_consolidado = pd.concat(lista_dfs, ignore_index=True)
    df_consolidado = df_consolidado.sort_values('ano')

    print("\n📊 TABELA FINAL:")
    print(df_consolidado[['ano', 'populacao']])

    df_consolidado.to_csv(os.path.join(PROCESSED_DIR, 'populacao_consolidada_DF.csv'), index=False)
else:
    print("\nNenhum dado gerado.")
resultado = processar_arquivo_dou('estimativa_dou_2021.xls', 2021)
print(resultado)

🚀 Iniciando consolidação...
--- Processando ano 2014: estimativa_dou_2014.xls ---
✅ 2014: Sucesso! População: 2852372
--- Processando ano 2015: estimativa_dou_2015.xls ---
✅ 2015: Sucesso! População: 2914830
--- Processando ano 2016: estimativa_TCU_2016_20170614.xls ---
✅ 2016: Sucesso! População: 2977216
--- Processando ano 2017: estimativa_dou_2017.xls ---
✅ 2017: Sucesso! População: 3039444
--- Processando ano 2018: estimativa_dou_2018_20181019.xls ---
✅ 2018: Sucesso! População: 2974703
--- Processando ano 2019: estimativa_dou_2019.xls ---
✅ 2019: Sucesso! População: 3015268
--- Processando ano 2020: estimativa_dou_2020.xls ---
✅ 2020: Sucesso! População: 3055149
--- Processando ano 2021: estimativa_dou_2021.xls ---
✅ 2021: Sucesso! População: 3094325
--- Processando ano 2024: estimativa_dou_2024.xls ---
✅ 2024: Sucesso! População: 2982818
--- Processando ano 2025: estimativa_dou_2025.xls ---
✅ 2025: Sucesso! População: 2996899

📊 TABELA FINAL:
    ano  populacao
0  2014    2852372